A função para decidir se é preciso trocar de caixa ou não é a seguinte:

In [ ]:
# The probability to win will always be higher if the player switches boxes, unless there is only one box to be opened,
# in which case, the remaining box is the one with the Apple Watch, but this is not a realistic scenario since the 
# player will be sure the watch is there.
# Fore more information, watch https://www.youtube.com/watch?v=LIyg28vXQdQ

def switch_boxes(total_boxes, open_boxes=0):

  if (total_boxes < open_boxes + 2):
    print("The game needs at least two closed boxes, one with the Apple Watch and one empty, to make sense.")
    return False
  
  p_win_before_open = 1 / total_boxes

  p_win_if_switch_after_open = (total_boxes - 1) / (total_boxes * (total_boxes - open_boxes - 1))
  
  # if p_win_if_switch_after_open == p_win_before_open, probabilistically speaking it doesn't make any difference
  # to switch the currently selected box or not. In this implementation, I decided to switch.
  if p_win_if_switch_after_open >= p_win_before_open:
    return True
  return False


Acontece que o modelo matemático que descreve este problema se chama problema de **Monty Hall**, e segundo sua solução, é sempre vantajoso trocar de caixa após uma ou mais caixas vazias serem reveladas. Vejamos alguns exemplos.

In [ ]:
print(switch_boxes(7, 5))

True


In [ ]:
print(switch_boxes(3, 1))

True


In [ ]:
print(switch_boxes(50, 1))

True


In [ ]:
print(switch_boxes(50, 48))

True


In [ ]:
print(switch_boxes(978787, 3574))

True


In [ ]:
print(switch_boxes(3, 0))

True


Abaixo, segue um programa que simula um problema de Monty Hall iterativo. O jogador e o condutor do jogo alternam entre decidir trocar de caixa e revelar uma caixa vazia, respectivamente.

É possível variar o número de caixas (variável **number_of_boxes**) para executar simulações mais longas, mas como vimos na função passada, independente do número de caixas presentes no jogo e quantas já foram reveladas, é sempre probabilisticamente vantajoso mudar sua escolha de caixa após pelo menos uma caixa vazia ser revelada.

In [ ]:
import sys
import random

random.seed()
number_of_boxes = int(input("Number of boxes in the game: "))

# Generate random boxes contents
boxes = ["empty box" for i in range(number_of_boxes - 1)] + ["apple watch"]

# Shuffle boxes
random.shuffle(boxes)
print("Boxes (in position): {}".format(boxes))

# Player pick first box
box_choice = random.randrange(number_of_boxes)
print("First box selected: {}".format(box_choice))

Number of boxes in the game: 20
Boxes (in position): ['empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'apple watch', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box']
First box selected: 10


In [ ]:
# Slightly slower solution:
# computing the next boxes to reveal and to swap to does require attempts, 
# but consumes less memory than the alternative with the available_boxes list
# -----------------------------------------------------------------------------
#
# Initialize auxiliary variables
apple_watch_position = boxes.index("apple watch")
revealed_boxes = []
box_to_reveal = random.randrange(number_of_boxes)
box_to_swap = random.randrange(number_of_boxes)

while len(revealed_boxes) < number_of_boxes - 2:

  # Host reveals a random empty box
  while box_to_reveal == box_choice or box_to_reveal in revealed_boxes or box_to_reveal == apple_watch_position:
    box_to_reveal = random.randrange(number_of_boxes)

  revealed_boxes.append(box_to_reveal)
  print("Empty box revelead: {}".format(box_to_reveal))

  # Player swap to another random, unrevealed box
  # According to the Monty Hall problem, a player should always switch boxes after the host reveals empty boxes.
  # This if statement is here only to additionally illustrate this behavior (no matter how many times this 
  # code runs, or how many boxes we set on the number_of_boxes variable, the else block will never run).
  if (switch_boxes(number_of_boxes, len(revealed_boxes))):
    while box_to_swap == box_choice or box_to_swap in revealed_boxes:
      box_to_swap = random.randrange(number_of_boxes)

    box_choice = box_to_swap
    print("Box swapped to {}".format(box_to_swap))
  else:
    print("Player did not swap boxes")
  
if boxes[box_choice] == "apple watch":
    print("You won the Apple Watch!")
else:
    print("You won nothing.")

Empty box revelead: 1
Box swapped to 3
Empty box revelead: 18
Box swapped to 17
Empty box revelead: 8
Box swapped to 12
Empty box revelead: 14
Box swapped to 3
Empty box revelead: 17
Box swapped to 0
Empty box revelead: 6
Box swapped to 15
Empty box revelead: 13
Box swapped to 9
Empty box revelead: 12
Box swapped to 19
Empty box revelead: 5
Box swapped to 2
Empty box revelead: 0
Box swapped to 3
Empty box revelead: 2
Box swapped to 15
Empty box revelead: 4
Box swapped to 7
Empty box revelead: 9
Box swapped to 3
Empty box revelead: 15
Box swapped to 7
Empty box revelead: 11
Box swapped to 16
Empty box revelead: 7
Box swapped to 19
Empty box revelead: 16
Box swapped to 10
Empty box revelead: 19
Box swapped to 3
You won nothing.


E uma versão um pouco mais rápida, mas que consome mais memória:

In [ ]:
# Faster Solution:
# computing the next boxes to reveal and to swap to does not require attempts, 
# but it consumes more memory because of the available_boxes list.
# -----------------------------------------------------------------------------
#
# Initialize auxiliary variables
apple_watch_position = boxes.index("apple watch")
revealed_boxes = []
available_boxes = [ i for i in range(len(boxes)) if i != apple_watch_position and i != box_choice ]

while len(revealed_boxes) < number_of_boxes - 2:

  # Host reveals a random empty box
  box_to_reveal = random.choice(available_boxes)
  revealed_boxes.append(box_to_reveal)
  available_boxes.remove(box_to_reveal)

  print("Empty box revelead: {}".format(box_to_reveal))

  # Player swap to another random, unrevealed box.
  # According to the Monty Hall problem, a player should always switch boxes after the host reveals empty boxes
  # This if statement is here only to additionally illustrate this behavior (no matter how many times this 
  # code runs, or how many boxes we set on the number_of_boxes variable, the else block will never run).
  if (switch_boxes(number_of_boxes, len(revealed_boxes))):
    box_to_swap = random.choice(available_boxes + [ apple_watch_position ])
    
    if box_choice != apple_watch_position:
      available_boxes.append(box_choice)

    if box_to_swap != apple_watch_position:
      available_boxes.remove(box_to_swap)

    box_choice = box_to_swap
    print("Box swapped to {}".format(box_to_swap))
  else:
    print("Player did not swap boxes")
  
if box_choice == apple_watch_position:
    print("You won the Apple Watch!")
else:
    print("You won nothing.")


Empty box revelead: 15
Box swapped to 7
Empty box revelead: 14
Box swapped to 8
Empty box revelead: 12
Box swapped to 10
Empty box revelead: 0
Box swapped to 16
Empty box revelead: 17
Box swapped to 19
Empty box revelead: 1
Box swapped to 16
Empty box revelead: 5
Box swapped to 9
Empty box revelead: 2
Box swapped to 4
Empty box revelead: 6
Box swapped to 19
Empty box revelead: 16
Box swapped to 18
Empty box revelead: 11
Box swapped to 19
Empty box revelead: 3
Box swapped to 8
Empty box revelead: 7
Box swapped to 18
Empty box revelead: 13
Box swapped to 9
Empty box revelead: 19
Box swapped to 10
Empty box revelead: 9
Box swapped to 10
Empty box revelead: 18
Box swapped to 4
Empty box revelead: 8
Box swapped to 10
You won the Apple Watch!
